In [ ]:
import mne
from mne.datasets import eegbci
from mne.io import read_raw_edf
import numpy as np
import pandas as pd
import tensorflow as tf
np.random.seed(42)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, BatchNormalization, Conv2D
from keras.layers import Reshape, Permute, ZeroPadding2D
from keras.layers import MaxPooling2D

In [ ]:
subjects = range(1,110)
runs = [1,2] # Baseline: Eyes open vs eyes closed

raw_fnames = list()
for subject in subjects: 
    raw_fnames.append(eegbci.load_data(subject, runs))
    
# We now need to flatten the list
flat_list = [item for sublist in raw_fnames for item in sublist]

raw_files = [read_raw_edf(f, preload=True, stim_channel='auto') for f in flat_list]

pd.set_option('display.max_colwidth', -1)

df = pd.DataFrame(raw_fnames)
df = df.rename(columns={0: "Eyes_Open", 1: "Eyes_Closed"})

raw_files_open = [read_raw_edf(f, preload=True, stim_channel='auto') for f in df.Eyes_Open]
raw_files_closed = [read_raw_edf(f, preload=True, stim_channel='auto') for f in df.Eyes_Closed]

# The events are spaced evenly every 1 second.
duration = 1.
event_id_o = 1 # Eyes open
event_id_c = 0 # Eyes closed

# for fixed size events no start time before and after event
tmin = 0.
tmax = 0.99  # inclusive tmax, 1 second epochs


# create a fixed size events array
# start=0 and stop=None by default


# Get all the open events

temp_o = []

for raw_ in raw_files_open:    
    events_o = mne.make_fixed_length_events(raw_, event_id_o, duration=duration)
    # create :class:`Epochs <mne.Epochs>` object
    epochs_o = mne.Epochs(raw_, events=events_o, event_id=event_id_o, tmin=tmin,
                        tmax=tmax, baseline=None, verbose=True)
    for ep in epochs_o:
        temp_o.append(ep)

big_raw_o = np.array(temp_o)

# Get all the open events


temp_c = []

for raw_ in raw_files_closed:    
    events_c = mne.make_fixed_length_events(raw_, event_id_c, duration=duration)
    # create :class:`Epochs <mne.Epochs>` object
    epochs_c = mne.Epochs(raw_, events=events_c, event_id=event_id_c, tmin=tmin,
                        tmax=tmax, baseline=None, verbose=True)
    for ep in epochs_c:
        temp_c.append(ep)

big_raw_c = np.array(temp_c)

labels_o = np.ones(len(big_raw_o))
labels_c = np.zeros(len(big_raw_c))

big_raw = np.concatenate((big_raw_o[:, :-1, :], big_raw_c[:, :-1, :]), axis=0)
big_labels = np.concatenate((labels_o, labels_c), axis=0)

# preprocess dataset, split into training and test part
X, y = big_raw, big_labels

X_3D_train, X_3D_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

X_3D_train = X_3D_train.reshape(X_3D_train.shape[0], 1, 64, 159)
X_3D_test = X_3D_test.reshape(X_3D_test.shape[0], 1, 64, 159)

y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

In [ ]:
# EEGnet model
EEGnet = Sequential()

# Layer 1
EEGnet.add(Conv2D(input_shape=(1,64,159), filters=16, kernel_size=(64, 1), data_format='channels_first'))
EEGnet.add(BatchNormalization(axis=1))
EEGnet.add(Permute((2,1,3), input_shape=(16,1,159)))
EEGnet.add(Dropout(.25))

# Padding to get N % 4
EEGnet.add(ZeroPadding2D(padding=((0,1), (16,17)), data_format="channels_first"))

# Layer 2
EEGnet.add(Conv2D(filters=4, kernel_size=(2, 32), activation='relu', data_format='channels_first'))
EEGnet.add(BatchNormalization(axis=1))
EEGnet.add(MaxPooling2D(pool_size=(2,4), data_format='channels_first'))
EEGnet.add(Dropout(0.25))

# Padding to get N % 4
EEGnet.add(ZeroPadding2D(padding=((4,3), (2,1)), data_format="channels_first"))

# Layer 3
EEGnet.add(Conv2D(filters=4, kernel_size=(8, 4), activation='relu', data_format='channels_first'))
EEGnet.add(BatchNormalization(axis=1))
EEGnet.add(MaxPooling2D(pool_size=(2,4), data_format='channels_first'))
EEGnet.add(Dropout(0.25))


# Layer 4
EEGnet.add(Flatten())
EEGnet.add(Dense(2, activation='softmax'))

EEGnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

EEGnet.fit(X_3D_train, y_train, 
          batch_size=100, epochs=500, verbose=1)

score = EEGnet.evaluate(X_3D_test, y_test, batch_size=32)

print(score)